<a href="https://colab.research.google.com/github/AJamal27891/University-coursees-projects/blob/master/XLnet_Base_Matching_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download dependencies

In [ ]:
!rm -r  test_trainer 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets

## load datasets 

In [ ]:
dirty_walmart_amazon = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_walmart_amazon'
dirty_dblp_scholar = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_scholar'
dirty_amazon_ituns = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_amazon_itunes'
dirty_dblp_acm = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_acm'
abt_buy = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/abt_buy'
amazon_google = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/amazon_google'
QQP= r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/QQP'

## Test dataset merger functions

In [ ]:
import pandas as pd 
data_A = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['samsung s5','iphone','xperia','sonny','hstc','xomi']})
data_B = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['iphone','xperia','sonny','huawi','xomi','samsung s5']})
lables = pd.DataFrame({'left_id':[1,2,3,4,5,6],'right_id':[6,1,2,3,4,5],'label':[1,1,1,1,0,1]})

In [ ]:
total_data = lables.merge(data_A,left_on='left_id',right_on='id')
total_data = total_data.merge(data_B,left_on='right_id',right_on='id')
total_data

,left_id,right_id,label,id_x,price_x,describtion_x,id_y,price_y,describtion_y
0,1,6,1,1,0.2,samsung s5,6,0.8,samsung s5
1,2,1,1,2,0.5,iphone,1,0.2,iphone
2,3,2,1,3,0.6,xperia,2,0.5,xperia
3,4,3,1,4,0.7,sonny,3,0.6,sonny
4,5,4,0,5,1.0,hstc,4,0.7,huawi
5,6,5,1,6,0.8,xomi,5,1.0,xomi


In [ ]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

,label,price_x,describtion_x,price_y,describtion_y
0,1,0.2,samsung s5,0.8,samsung s5
1,1,0.5,iphone,0.2,iphone
2,1,0.6,xperia,0.5,xperia
3,1,0.7,sonny,0.6,sonny
4,0,1.0,hstc,0.7,huawi
5,1,0.8,xomi,1.0,xomi


In [ ]:
total_data =remove_id_columns(total_data)

In [ ]:
total_data.astype(str).apply(lambda x : ' <atr>'+x.name.strip('_y').strip('_x') +'<val>'+x)

,label,price_x,describtion_x,price_y,describtion_y
0,<atr>label<val>1,<atr>price<val>0.2,<atr>describtion<val>samsung s5,<atr>price<val>0.8,<atr>describtion<val>samsung s5
1,<atr>label<val>1,<atr>price<val>0.5,<atr>describtion<val>iphone,<atr>price<val>0.2,<atr>describtion<val>iphone
2,<atr>label<val>1,<atr>price<val>0.6,<atr>describtion<val>xperia,<atr>price<val>0.5,<atr>describtion<val>xperia
3,<atr>label<val>1,<atr>price<val>0.7,<atr>describtion<val>sonny,<atr>price<val>0.6,<atr>describtion<val>sonny
4,<atr>label<val>0,<atr>price<val>1.0,<atr>describtion<val>hstc,<atr>price<val>0.7,<atr>describtion<val>huawi
5,<atr>label<val>1,<atr>price<val>0.8,<atr>describtion<val>xomi,<atr>price<val>1.0,<atr>describtion<val>xomi


In [ ]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

def merger(data_A,data_B,train,test,valid,
           remove_ids=True,
           merge_attributes=True,
           drop_original=True,
           tuples=True):
  train_data = train.merge(data_A,left_on='ltable_id',right_on='id')
  train_data = train_data.merge(data_B,left_on='rtable_id',right_on='id')
  test_data = test.merge(data_A,left_on='ltable_id',right_on='id')
  test_data = test_data.merge(data_B,left_on='rtable_id',right_on='id')
  valid_data = valid.merge(data_A,left_on='ltable_id',right_on='id')
  valid_data = valid_data.merge(data_B,left_on='rtable_id',right_on='id')
  if remove_ids:
    train_data = remove_id_columns(train_data)
    test_data = remove_id_columns(test_data)
    valid_data = remove_id_columns(valid_data)
  if merge_attributes:
    train_data.iloc[:,1:]  = train_data.iloc[:,1:].astype(str).apply(lambda x : ' <atr>'+x.name.strip('_y').strip('_x') +'<val>'+x)
    train_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x :"<cls>" + ' '.join(x.astype(str)) ,axis=1)
    train_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x :"<sep>" + ' '.join(x.astype(str)) ,axis=1)
    test_data.iloc[:,1:]  = test_data.iloc[:,1:].astype(str).apply(lambda x : ' <atr>'+x.name.strip('_y').strip('_x') +'<val>'+x)
    test_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x :"<cls>" + ' '.join(x.astype(str)) ,axis=1)
    test_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x :"<sep>" + ' '.join(x.astype(str)) ,axis=1)
    valid_data.iloc[:,1:]  = valid_data.iloc[:,1:].astype(str).apply(lambda x : ' <atr>'+x.name.strip('_y').strip('_x') +'<val>'+x)
    valid_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x :"<cls>" + ' '.join(x.astype(str)) ,axis=1)
    valid_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x :"<sep>" + ' '.join(x.astype(str)) ,axis=1) 

  if drop_original:
    train_data= train_data[['label','text_left','text_right']]
    test_data= test_data[['label','text_left','text_right']]
    valid_data= valid_data[['label','text_left','text_right']]
  if tuples:
    train_data['tuples']= train_data[['text_left','text_right']].values.tolist()
    train_data['tuples'] = train_data['tuples'].apply(lambda x: '<sep>'.join(x))
    test_data['tuples']= test_data[['text_left','text_right']].values.tolist()
    test_data['tuples'] = test_data['tuples'].apply(lambda x: '<sep>'.join(x))
    valid_data['tuples']= valid_data[['text_left','text_right']].values.tolist()
    valid_data['tuples'] = valid_data['tuples'].apply(lambda x: '<sep>'.join(x))

  return train_data, test_data, valid_data

# Data Preprocessing

## Amazon Google data

In [ ]:
AG_train = pd.read_csv(amazon_google+r'/deep_matcher/train.csv')


In [ ]:
table_A = pd.read_csv(amazon_google+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(amazon_google+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(amazon_google+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(amazon_google+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(amazon_google+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

,label,text_left,text_right,tuples
0,0,<cls> <atr>title<val>microsoft visio standard ...,<sep> <atr>title<val>adobe cs3 design standard...,<cls> <atr>title<val>microsoft visio standard ...
1,0,<cls> <atr>title<val>adobe premiere pro cs3 up...,<sep> <atr>title<val>adobe cs3 design standard...,<cls> <atr>title<val>adobe premiere pro cs3 up...
2,0,<cls> <atr>title<val>adobe indesign cs3 upgrad...,<sep> <atr>title<val>adobe cs3 design standard...,<cls> <atr>title<val>adobe indesign cs3 upgrad...
3,0,<cls> <atr>title<val>adobe creative suite cs3 ...,<sep> <atr>title<val>adobe cs3 design standard...,<cls> <atr>title<val>adobe creative suite cs3 ...
4,0,<cls> <atr>title<val>adobe creative suite cs3 ...,<sep> <atr>title<val>adobe cs3 design standard...,<cls> <atr>title<val>adobe creative suite cs3 ...


In [ ]:
AG_train['tuples'][0]

'<cls> <atr>title<val>microsoft visio standard 2007 version upgrade  <atr>manufacturer<val>microsoft  <atr>price<val>129.95<sep><sep> <atr>title<val>adobe cs3 design standard upgrade  <atr>manufacturer<val>nan  <atr>price<val>413.99'

In [ ]:
AG_train['label'].describe()

count    6874.000000
mean        0.101688
std         0.302259
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [ ]:
AG_train = AG_train.groupby('label')
AG_train = AG_train.apply(lambda x: x.sample(AG_train.size().min()).reset_index(drop=True))
AG_train = AG_train.sample(frac=1).reset_index(drop=True)

In [ ]:
import os 
try:
  os.mkdir('merged_data')
  os.mkdir('merged_data/amazon_google')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/amazon_google/train.csv')
AG_test.to_csv(r'merged_data/amazon_google/test.csv')
AG_valid.to_csv(r'merged_data/amazon_google/valid.csv')



In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':'merged_data/amazon_google/train.csv',
                                              'test':'merged_data/amazon_google/test.csv',
                                              'valid':'merged_data/amazon_google/valid.csv'})

Using custom data configuration default-a9b681e588aa8853


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a9b681e588aa8853/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

In [ ]:
maxlen = 128
def tokenize_function(examples):
    return tokenizer(examples['tuples'], padding="max_length", truncation=True,max_length=maxlen)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 1398
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 2293
    })
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 2293
    })
})

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",num_train_epochs=60,per_device_train_batch_size=16)

In [ ]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][1])

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><cls> <atr>title<val>punch! master landscape & home design <atr>manufacturer<val>punch! software <atr>price<val>99.99<sep><sep> <atr>title<val>punch software 25100 punch! master landscape & home design architectural 1 user ( s ) pc <atr>manufacturer<val>nan <atr>price<val>45.97<sep><cls>'

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)

In [26]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text_left, text_right, tuples.
***** Running training *****
  Num examples = 1398
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8800


Step,Training Loss
500,0.330000
1000,0.133400
1500,0.109500
2000,0.085300
2500,0.074100
3000,0.075300
3500,0.061600


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

Step,Training Loss
500,0.330000
1000,0.133400
1500,0.109500
2000,0.085300
2500,0.074100
3000,0.075300
3500,0.061600
4000,0.053800
4500,0.048800
5000,0.045700


Saving model checkpoint to test_trainer/checkpoint-4000
Configuration saved in test_trainer/checkpoint-4000/config.json
Model weights saved in test_trainer/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-4500
Configuration saved in test_trainer/checkpoint-4500/config.json
Model weights saved in test_trainer/checkpoint-4500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-5000
Configuration saved in test_trainer/checkpoint-5000/config.json
Model weights saved in test_trainer/checkpoint-5000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-5500
Configuration saved in test_trainer/checkpoint-5500/config.json
Model weights saved in test_trainer/checkpoint-5500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-6000
Configuration saved in test_trainer/checkpoint-6000/config.json
Model weights saved in test_trainer/checkpoint-6000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint

TrainOutput(global_step=8800, training_loss=0.06773504620248622, metrics={'train_runtime': 4818.4018, 'train_samples_per_second': 29.014, 'train_steps_per_second': 1.826, 'total_flos': 1.25952024167424e+16, 'train_loss': 0.06773504620248622, 'epoch': 100.0})

In [27]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [28]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][0])

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><cls> <atr>title<val>cosmi rom07524 print perfect business cards dvd <atr>manufacturer<val>cosmi <atr>price<val>nan<sep><sep> <atr>title<val>encore inc printshop business card maker <atr>manufacturer<val>riverdeep-learning-company <atr>price<val>8.01<sep><cls>'

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text_left, text_right, tuples.
***** Running Evaluation *****
  Num examples = 2293
  Batch size = 8


{'eval_f1': 0.07025761124121781,
 'eval_loss': 1.3468997478485107,
 'eval_runtime': 26.1569,
 'eval_samples_per_second': 87.663,
 'eval_steps_per_second': 10.972}

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text_left, text_right, tuples.
***** Running Evaluation *****
  Num examples = 2293
  Batch size = 8


{'eval_accuracy': 0.8268643698211949,
 'eval_loss': 1.3468997478485107,
 'eval_runtime': 26.1268,
 'eval_samples_per_second': 87.764,
 'eval_steps_per_second': 10.985}